In [1]:
import json

major_voting_results = json.load(open("birddev-generated_sql_eval_results.json", "r"))
upper_bound_results = json.load(open("birddev-generated_sql_eval_upper_bound.json", "r"))

diff_results = []
for res1, res2 in zip(major_voting_results, upper_bound_results):
    if res1["question_id"] != res2["question_id"]:
        print(res1["question_id"])
    elif res1["correctness"] != res2["correctness"]:
        res = {}
        res["question_id"] = res1["question_id"]
        res["max_vote_correctness"] = res1["correctness"]
        res["upper_bound_correctness"] = res2["correctness"]
        res["ground_truth"] = res1["ground_truth"]
        res["max_vote_sql"] = res1["pred_sql"]
        res["upper_bound_sql"] = res2["pred_sql"]
        diff_results.append(res)

print(len(diff_results))
# with open("birddev-generated_sql_eval_diff.json", "w") as f:
#     json.dump(diff_results, f, indent=2, ensure_ascii=False)

157


In [2]:
import json

results = json.load(open("birddev-generated_sql_analysis_diff.json", "r"))
print(len(results))
qid_to_index = {}
for i, res in enumerate(results):
    qid_to_index[res["question_id"]] = i

157


In [3]:
import json
from collections import defaultdict

test_data = json.load(open("/ssddata/wushw/text2sql/SQL-R1/data/NL2SQL/BIRD/dev/dev.json"))
# categorize the test_data based on db_id and difficulty
categorized_data = defaultdict(list)
for d in test_data:
    db_id = d["db_id"]
    difficulty = d["difficulty"]
    if d["question_id"] in qid_to_index:
        index = qid_to_index[d["question_id"]]
        categorized_data[f"{db_id}_{difficulty}"].append(index)

print(len(categorized_data))

29


In [4]:
categories = list(categorized_data.keys())
print(categories[:10])

['california_schools_moderate', 'california_schools_simple', 'financial_simple', 'financial_moderate', 'toxicology_simple', 'toxicology_challenging', 'toxicology_moderate', 'card_games_moderate', 'card_games_simple', 'codebase_community_simple']


In [5]:
for cate, qids in categorized_data.items():
    print(cate, qids)


california_schools_moderate [0, 1, 5, 6, 7, 8]
california_schools_simple [2, 3, 4, 9, 10, 11, 12, 13, 14]
financial_simple [15, 18, 19, 21, 24, 25, 26, 29, 30]
financial_moderate [16, 17, 20, 22, 23, 27, 28, 31, 32, 33]
toxicology_simple [34, 35, 36, 37, 40, 41, 47, 48, 49]
toxicology_challenging [38, 42, 45]
toxicology_moderate [39, 43, 44, 46]
card_games_moderate [50, 51, 57, 58, 60]
card_games_simple [52, 53, 54, 55, 56, 59]
codebase_community_simple [61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 73, 74, 75, 76, 78, 79, 80, 81]
codebase_community_moderate [71, 77]
codebase_community_challenging [72]
superhero_challenging [82]
superhero_simple [83, 86, 87, 88]
superhero_moderate [84, 85]
formula_1_simple [89, 90, 91, 94, 95, 96, 97, 98, 99, 100, 101, 102, 106, 107, 109]
formula_1_moderate [92, 93, 103]
formula_1_challenging [104, 105, 108, 110]
european_football_2_simple [111, 114, 119, 121, 122]
european_football_2_moderate [112, 115, 116, 117, 118, 120]
european_football_2_challenging [1

In [15]:
idx = 156

print(results[idx]["question_id"])
print("--------------------------------")
print("ERROR SQL:")
print(results[idx]["max_vote_sql"])
print("--------------------------------")
print("CORRECT SQL:")
print(results[idx]["upper_bound_sql"])
print("--------------------------------")
print("ERROR ANALYSIS:")
print(results[idx]["error_analysis"])

1526
--------------------------------
ERROR SQL:
-- Your SQL query
--------------------------------
CORRECT SQL:
-- Step 1: Find the CustomerID of the customer who made the transaction of 634.8 on '2012-08-25'
WITH CustomerID AS (
    SELECT CustomerID
    FROM transactions_1k
    WHERE Date = '2012-08-25' AND Price = 634.8
)

-- Step 2: Join the transactions_1k table with the yearmonth table and calculate the consumption decrease rate
SELECT 
    (SUM(CASE WHEN substr(y.`Date`, 1, 4) = '2012' THEN y.Consumption ELSE 0 END) -
     SUM(CASE WHEN substr(y.`Date`, 1, 4) = '2013' THEN y.Consumption ELSE 0 END)) /
    SUM(CASE WHEN substr(y.`Date`, 1, 4) = '2012' THEN y.Consumption ELSE 0 END) AS ConsumptionDecreaseRate
FROM transactions_1k t
JOIN yearmonth y ON t.CustomerID = y.CustomerID
JOIN CustomerID c ON t.CustomerID = c.CustomerID;
--------------------------------
ERROR ANALYSIS:
<think>
让我分析这两个SQL的差异：

正确SQL的逻辑：
1. 首先通过CTE找到在2012-08-25日进行634.8金额交易的CustomerID
2. 然后通过三表连接（transactions

In [ ]:
import json 

org_file = "birddev-generated_sql.json"
new_file = "birddev-sql_14B.json"

org_results = json.load(open(org_file, "r"))
qid_sqls = {}
for d in org_results:
    qid = d["question_id"]
    sqls = d["pred_sqls"]
    qid_sqls[qid] = sqls

with open(new_file, "w") as f:
    json.dump(qid_sqls, f, indent=2, ensure_ascii=False)